In [1]:
import pythran
%load_ext pythran.magic

In [2]:
def get_complex_random_array(dimensions):
    import numpy as np
    real_part = np.random.rand(*dimensions)
    imaginary_part = np.random.rand(*dimensions)
    complex_array = real_part + imaginary_part * 1.0j
    return complex_array

In [10]:
import numpy as np
from numba import complex64, prange, jit, types

@jit(['complex64[:, :, :] (float32[:], float32[:], float32[:], \
                                        complex64[:, :], complex64[:, :], complex64[:, :])',
      'complex128[:, :, :] (float64[:], float64[:], float64[:], \
                                        complex128[:, :], complex128[:, :], complex128[:, :])'],
     nopython=True, cache=True, nogil=True, parallel=True, fastmath=True)
def numba_tensordots(x_float_1dim, y_float_1dim, z_float_1dim, \
                        x_complex_2dim, y_complex_2dim, z_complex_2dim):

    first_shape = x_float_1dim.shape
    second_shape = x_complex_2dim.shape
    result = np.zeros((first_shape[0], second_shape[0], second_shape[1]), dtype=type(x_complex_2dim[0, 0]))
    for i in prange(first_shape[0]):
        for j in range(second_shape[0]):
            for k in range(second_shape[1]):
                result[i, j, k] = x_float_1dim[i] * x_complex_2dim[j, k] + \
                                    y_float_1dim[i] * y_complex_2dim[j, k] + \
                                        z_float_1dim[i] * z_complex_2dim[j, k]

    return result

In [4]:
%%pythran -fopenmp -Ofast
#pythran export pythran_tensordots(float32[:], float32[:], float32[:], complex64[:, :], complex64[:, :], complex64[:, :]))

def pythran_tensordots(x_float_1dim, y_float_1dim, z_float_1dim, \
                        x_complex_2dim, y_complex_2dim, z_complex_2dim):
    import numpy as np
    first_shape = x_float_1dim.shape
    second_shape = x_complex_2dim.shape
    result = np.zeros((first_shape[0], second_shape[0], second_shape[1]), dtype=type(x_complex_2dim[0, 0]))
    "omp parallel for"
    for i in range(first_shape[0]):
        for j in range(second_shape[0]):
            for k in range(second_shape[1]):
                result[i, j, k] = x_float_1dim[i] * x_complex_2dim[j, k] + \
                                    y_float_1dim[i] * y_complex_2dim[j, k] + \
                                        z_float_1dim[i] * z_complex_2dim[j, k]

    return result

In [5]:
def numpy_tensordots(x_float_1dim, y_float_1dim, z_float_1dim, \
                        x_complex_2dim, y_complex_2dim, z_complex_2dim):
    result = np.tensordot(x_float_1dim, x_complex_2dim, axes=0)
    result += np.tensordot(y_float_1dim, y_complex_2dim, axes=0)
    result += np.tensordot(z_float_1dim, z_complex_2dim, axes=0)
    
    return result

In [11]:
import datetime
import numpy as np

def test_tensordots():
    numpy_time = []
    pythran_time = []
    numba_time = []
    for i in range(100):
        float_1dim_x = np.random.rand(50).astype(np.float32)
        float_1dim_y = np.random.rand(50).astype(np.float32)
        float_1dim_z = np.random.rand(50).astype(np.float32)
        complex_2dim_x = get_complex_random_array((300, 710)).astype(np.complex64)
        complex_2dim_y = get_complex_random_array((300, 710)).astype(np.complex64)
        complex_2dim_z = get_complex_random_array((300, 710)).astype(np.complex64)

        #numpy test zone
        numpy_start = datetime.datetime.now()
        res = numpy_tensordots(float_1dim_x, float_1dim_y, float_1dim_z, \
                        complex_2dim_x, complex_2dim_y, complex_2dim_z)
        numpy_end = datetime.datetime.now()
        numpy_time.append(numpy_end - numpy_start)

        #pythran test zone
        pythran_start = datetime.datetime.now()
        res = pythran_tensordots(float_1dim_x, float_1dim_y, float_1dim_z, \
                        complex_2dim_x, complex_2dim_y, complex_2dim_z)
        pythran_end = datetime.datetime.now()
        pythran_time.append(pythran_end - pythran_start)
        
        #numba test zone
        numba_start = datetime.datetime.now()
        res = numba_tensordots(float_1dim_x, float_1dim_y, float_1dim_z, \
                        complex_2dim_x, complex_2dim_y, complex_2dim_z)
        numba_end = datetime.datetime.now()
        numba_time.append(numba_end - numba_start)
        
    print('numpy time: ', np.sum(numpy_time))
    print('pythran time: ', np.sum(pythran_time))
    print('numba time: ', np.sum(numba_time))

    print('pythran speed up = ', np.sum(numpy_time) / np.sum(pythran_time))
    print('numba speed up = ', np.sum(numpy_time) / np.sum(numba_time))
    
    
test_tensordots()

numpy time:  0:00:08.694998
pythran time:  0:00:02.668248
numba time:  0:00:06.040465
pythran speed up =  3.2586918457354788
numba speed up =  1.43945838606796
